Copyright © 2021, SAS Institute Inc., Cary, NC, USA.  All Rights Reserved. 

# Configure a Git Publishing Destination

In order to create a Git destination, you must complete the following steps:

1. Get an authorization token. 
2. Create a domain or find a valid domain with Git credentials.
3. Create credentials for a specific user or group and define a credential domain.
4. Submit an API post to define and create a Git destination.

   _**Note**: An example of an API request to delete a destination is also included._

In [ ]:
import requests
import json, os, pprint
import getpass
import base64
import urllib.parse

hostport="<host_name:port>"

## Get an Authorization Token and Define Headers

In [ ]:
#Get Authorization Token
authUri="/SASLogon/oauth/token"

headersAuth={
    "accept":"application/json",
    "content-type":"application/x-www-form-urlencoded",
    "Authorization":"Basic c2FzLmVjOg=="
}
authToken=""
user=""
password=""
notAuthed=True

while notAuthed :
    user = input("Enter user ID: ")
    password =  urllib.parse.quote(getpass.getpass('Enter password for user %s:' % user))
    authBody='grant_type=password&username=' + user + '&password=' + password
    authReturn = requests.post(hostport+authUri, data=authBody, headers=headersAuth, verify=False)
    if authReturn.status_code == requests.codes.ok :
        authToken = authReturn.json()['access_token']
        notAuthed = False
    else :
        print("Please enter a valid user ID and password.")
    
password = ""

In [ ]:
headersGet = {
    'Authorization': 'Bearer ' + authToken
}

## Create a Domain

In [ ]:
# Create a Domain
domains_uri = "/credentials/domains/"
credential_domain_headers = {
    "If-Match":"false",
    "Content-Type":"application/json",
    'Authorization': 'Bearer ' + authToken
}

# The domain name can be any easy to remember name. For example: gitdomain
domain_name = input("Enter a domain name: ")
domain_description = input("Enter a domain description: ")

my_domain_url = hostport + domains_uri + domain_name
domain_attrs = {
    "id":domain_name,
    "type":"base64",
    "description": domain_description
}

domain = requests.put(my_domain_url, 
                      data=json.dumps(domain_attrs), 
                      headers=credential_domain_headers, verify=False)

print(domain)

## Get a List of Domains to Select a Domain From

In [ ]:
# This code example enables a user to get a list of domains to select a domain from.

domains_uri_limit = domains_uri + "?limit=100"
domains = requests.get(hostport + domains_uri_limit, headers=headersGet)

print(domains)
for i, domain in enumerate(domains.json()["items"]):
    print("%3d. domain name : %s " % (i, domain["id"]))
itemNotSelected = True
while itemNotSelected : 
    domainIndex = input("Enter the index number to select a domain: ")
    try:
        domain_name = domains.json()["items"][int(domainIndex)]["id"]
        itemNotSelected = False
    except:
        print("Please enter the index number for a domain to select it from the list.")
print(domain_name)

## Create User Credential with a Git Access Token

_**Note:** Gitlab is used in this example to show how to obtain a Git access token._

1. Sign into your Gitlab account. 
2. Click the user icon on the right side of the toolbar and select **Settings** from the drop-down menu.
3. In the left panel, click **Access Tokens**.
4. Enter a name for the Git token.
5. Specify a date for **Expires at**.
6. Select the **write_repository** option.
7. Click **Create personal access token**.
8. Copy or write down the token name. 
   
   _**Important**: This token only shows once and cannot be retrieved later._

In [ ]:
credential_user_headers = {
    "If-Match":"false",
    "Content-Type":"application/json",
    'Authorization': 'Bearer ' + authToken
}

# User credential name is the user ID on a SAS Viya system.
user_credential_name = input("Enter the user credential name: ")
my_credential_url = hostport + domains_uri + domain_name + "/users/" + user_credential_name

# User ID in Git
gitUserId = input("Enter your Git user ID: ")

gitAccessToken = input("Enter the name of the Git access token: ")

encoded_userId = str(base64.b64encode(gitUserId.encode("utf-8")), "utf-8")
encoded_password = str(base64.b64encode(gitAccessToken.encode("utf-8")), "utf-8")

credential_attrs = {
    "domainId":domain_name,
    "identityType":"user",
    "identityId":user_credential_name,
    "domainType":"base64",
    "properties":{"gitUserId":encoded_userId},
    "secrets":{"gitAccessToken":encoded_password}
}

credential = requests.put(my_credential_url,
                          data=json.dumps(credential_attrs),
                          headers=credential_user_headers, verify=False)

print(credential)
pprint.pprint(credential.json())

## Get a Domain Credential

In [ ]:
credential_uri = "/credentials/domains/"+domain_name+"/credentials"
credentials = requests.get(hostport + credential_uri, headers=headersGet)

print(credentials)
for i, cred in enumerate(credentials.json()["items"]):
    print("%3d. credential name : %s " % (i, cred["identityId"]))
itemNotSelected = True
while itemNotSelected : 
    credIndex = input("Enter the index number to select a credential: ")
    try:
        cred_name = credentials.json()["items"][int(credIndex)]["identityId"]
        itemNotSelected = False
    except:
        print("Please enter an index number to select a credential from the list.")
print(cred_name)

## Define a Git Destination

_**Note:** After you create a domain and store the credential information for accessing your cloud provider, you can use this Jupyter notebook to define and create a publishing destination. You can also use SAS Environment Manager to define and create a Git publishing destination._

In [ ]:
destName = input("Enter a destination name: ")
destDescription = input('Enter a description for the destination %s: ' % destName)
userEmail = input("Enter your user email address for Git: ")
remoteRepositoryURL = input("Enter the URL for the Git Repository: ")
gitBranch = input("Enter the Git branch: ")

# The codeGenerationMode property is currently only used when decisions or rule sets are published to a Git destination. 
# The default value is "MAS", if a value is not specified.
codeGenerationMode = input("Enter a value of MAS or CAS for the code generation mode: ")

localRepositoryLocation = "/mmprojectpublic"

targetDestination={
    "name":destName,
    "destinationType":"git",
    "description":destDescription,
    "properties":[
        {"name": "remoteRepositoryURL",
        "value": remoteRepositoryURL},
        {"name": "gitBranch",
        "value": gitBranch},
        {"name": "localRepositoryLocation",
        "value": localRepositoryLocation},
        {"name": "userEmail",
        "value": userEmail},
        {"name": "credDomainID",
        "value": domain_name},
        {"name": "codeGenerationMode",
        "value": codeGenerationMode}]
}


## Create a New Destination

In [ ]:
headersPost = {
    "Content-Type":"application/vnd.sas.models.publishing.destination.git+json",
    'Authorization': 'Bearer ' + authToken
}

destination = requests.post(hostport + "/modelPublish/destinations", 
                       data=json.dumps(targetDestination), headers=headersPost, verify=False)
print(destination)
pprint.pprint(destination.json())

## Get the New Destination

In [ ]:
destination = requests.get(hostport + "/modelPublish/destinations/"+destName, headers=headersGet, verify=False)
print(destination)
pprint.pprint(destination.json())

## Delete a Destination

In [ ]:
# The destName line is only needed if you come back later after your session has ended.
# destName = input("Enter a destination name: ")
headersDelete={
    'Authorization': 'Bearer ' + authToken
}
deleteDestination = requests.delete(hostport + "/modelPublish/destinations/" + destName, headers = headersDelete)
print(deleteDestination)